## Markstammdatenregister: Data Preparation and Visualization

In [46]:
import os
import pandas as pd
import warnings
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import json
warnings.filterwarnings('ignore')

In [47]:
# Define the columns to import and their data types
SELECTED_COLS = [
    'MaStR-Nr. der Einheit',
    'Betriebs-Status',
    'Energieträger',
    'Bruttoleistung der Einheit',
    'Nettonennleistung der Einheit',
    'Bundesland',
    'Postleitzahl',
    'Straße',
    'Hausnummer',
    'Koordinate: Breitengrad (WGS84)',
    'Koordinate: Längengrad (WGS84)',
    'Inbetriebnahmedatum der Einheit',
    'Anzahl der Solar-Module',
    'Hauptausrichtung der Solar-Module',
    'Lage der Einheit',
    'Name des Anlagenbetreibers (nur Org.)'
]

dtype_dict = {
    'MaStR-Nr. der Einheit': str,
    'Betriebs-Status': str,
    'Energieträger': str,
    'Bruttoleistung der Einheit': float,
    'Nettonennleistung der Einheit': float,
    'Bundesland': str,
    'Postleitzahl': 'Int64',  
    'Straße': str,
    'Hausnummer': str,
    'Koordinate: Breitengrad (WGS84)': float,
    'Koordinate: Längengrad (WGS84)': float,
    'Inbetriebnahmedatum der Einheit': str,
    'Anzahl der Solar-Module': 'Int64',
    'Hauptausrichtung der Solar-Module': str,
    'Lage der Einheit': str,
    'Name des Anlagenbetreibers (nur Org.)': str
}

In [ ]:
# Pfad zum Verzeichnis
input_directory = 'data_raw/'
output_file = 'data_raw_merged.csv'

# Liste alle CSV-Dateien im Verzeichnis auf
csv_files = sorted([file for file in os.listdir(input_directory) if file.endswith('.csv')])

# Initialisiere eine Liste, um alle DataFrames zu speichern
dataframes_list = []

# Durchlaufe alle CSV-Dateien und verarbeite jede
for csv_file in csv_files:
    file_path = os.path.join(input_directory, csv_file)
    try:
        # Lese die CSV-Datei
        data = pd.read_csv(file_path, usecols=SELECTED_COLS, dtype=dtype_dict, delimiter=';', decimal=',', on_bad_lines='skip')
        print(data.shape)

        # Füge den DataFrame zur Liste hinzu
        dataframes_list.append(data)
    except Exception as e:
        print(f"Fehler beim Lesen von {csv_file}: {e}")

# Verbinde alle DataFrames in der Liste zu einem einzigen DataFrame
df_original = pd.concat(dataframes_list, ignore_index=True)

# Ausgabe der Gesamtanzahl der Zeilen des zusammengeführten DataFrames
print(f"Gesamtanzahl der Zeilen nach dem Zusammenführen: {len(df_original)}")

df_original.to_csv(output_file, index=False, sep = ";")

### Explorative Daten Analyse

In [ ]:
# Berechne die Häufigkeit der Energieträger
value_counts = df_original['Energieträger'].value_counts()

# Berechne die prozentualen Anteile
percentage = (value_counts / value_counts.sum()) * 100

# Selektiere die Top 10 Energieträger
top_10 = percentage.head(10)

print(top_10)

# Erstelle das Säulendiagramm
plt.figure(figsize=(20, 10))
ax = top_10.plot(kind='bar', color='skyblue')
plt.title('Top 10 Energieträger im Marktstammdatenregister')
plt.xlabel('Energieträger')
plt.ylabel('Prozent')
plt.xticks(rotation=45)
plt.yticks(range(0, 101, 10), [f'{i}%' for i in range(0, 101, 10)])  # Setzt die Y-Achse in 10% Schritten

# Füge die absoluten Zahlen über die Balken
for index, bar in enumerate(ax.patches):
    height = bar.get_height()
    label = f'{value_counts[top_10.index[index]]:,}'  # Formatierung mit Tausendertrennzeichen
    ax.text(bar.get_x() + bar.get_width() / 2, height + 0.5, label, ha='center')

# Save figure for Paper
plt.savefig("Verteilung_Energieträger.png", bbox_inches='tight')

plt.show()

### Data Preparation and Cleaning

#### Data Cleaning

In [ ]:
# Löschen von NaNs relevanter Spalten
before = df_original.shape[0]
filtered_data = df_original.dropna(subset=["Inbetriebnahmedatum der Einheit", "Postleitzahl", "Name des Anlagenbetreibers (nur Org.)", "Bundesland"])
print("Gelöschte Einträge: ", before - filtered_data.shape[0])

In [ ]:
# Filtern nach relevanten Einträgen
before = filtered_data.shape[0]
filtered_data = filtered_data[
        (filtered_data['Betriebs-Status'].isin(['In Betrieb', 'In Planung'])) &
        (filtered_data['Energieträger'] == 'Solare Strahlungsenergie')
    ]
print("Gelöschte Einträge: ", before - filtered_data.shape[0])

In [ ]:
# Filter Data
print(f"Shape Originaldatensatz: {df_original.shape}")
print(f"Shape gefilterter Datensatz: {filtered_data.shape}")
print(f"Anzahl gelöschte Zeilen: {df_original.shape[0] - filtered_data.shape[0]}")

#### Data Preparation

In [6]:
# Neue Spalte Betreiber erstellen
df = filtered_data
df["Betreiber"] = df['Name des Anlagenbetreibers (nur Org.)'].apply(
    lambda x: "natürliche Person" if isinstance(x, str) and "natürliche Person" in x else x)
df['Betreiber'] = df['Betreiber'].replace({
    'Lidl Vertriebs GmbH & Co. KG': 'Lidl Vertriebs-GmbH & Co. KG'
})

In [7]:
# Konvertieren von Inbetriebnahmedatum zu Datum und Extraktion des Jahres
df['Inbetriebnahmedatum der Einheit formatted'] = pd.to_datetime(df['Inbetriebnahmedatum der Einheit'], format= "%d/%m/%Y")
df['Year'] = df['Inbetriebnahmedatum der Einheit formatted'].dt.year.astype('Int64')

In [8]:
# Add leading zero if PLZ only has 4 chars
df["Postleitzahl"] = df["Postleitzahl"].astype(float).astype(int)
df["Postleitzahl"] = df["Postleitzahl"].astype(str)
df["Postleitzahl"] = df["Postleitzahl"].apply(lambda x: x.zfill(5) if len(x) == 4 else x)


In [ ]:
# Check if successful
test = df["Postleitzahl"].to_list()
print(len(test) == df.shape[0])

for plz in test:
    if len(plz) != 5:
        print(plz)

### Knapp 95% der Einträge haben keine Koordinaten
Daher approximieren wir die Koordinaten auf Basis der PLZ mithilfe einer Mapping Tabelle, welche Koordinaten für alle PLZ in Deutschland zur Verfügung stellt

In [ ]:
nas = df["Koordinate: Breitengrad (WGS84)"].isna().sum()
nas / df.shape[0]

In [ ]:
nas = df["Straße"].isna().sum()
nas / df.shape[0]

### Approximieren der Koordinaten mit PLZ

In [12]:
## Define Storage and different NA-Values in data
STORAGE_DIR = "PLZ-Mapping/"
NA_VALUES = [
    "",
    "#N/A",
    "#N/A N/A",
    "#NA",
    "-1.#IND",
    "-1.#QNAN",
    "-NaN",
    "-nan",
    "1.#IND",
    "1.#QNAN",
    "<NA>",
    "N/A",
    "NULL",
    "NaN",
    "n/a",
    "nan",
    "null",
]

In [13]:
## Define Function
def get_data(country: str) -> pd.DataFrame:
    """Load the data from disk; otherwise download and save it"""

    data_path = os.path.join(STORAGE_DIR, country.upper() + ".txt")
    zip_codes_csv = os.path.join(STORAGE_DIR, "zip_codes.csv")

    if os.path.exists(data_path):
        data = pd.read_csv(
            data_path,
            dtype={"postal_code": str},
            na_values=NA_VALUES,
            keep_default_na=False,
        )
        print("Data_path exists")

    if os.path.exists(zip_codes_csv):
        additional_data = pd.read_csv(zip_codes_csv, sep=";")
        print("Additional data loaded from CSV.")
        data['postal_code'] = data['postal_code'].astype(str)
        additional_data['postal_code'] = additional_data['postal_code'].astype(str)

        print("Vorher: ", data.shape[0])

        # Use an outer join to make sure all postal codes are included
        data = pd.merge(data, additional_data, on='postal_code', how='outer', suffixes=('', '_add'))

        # For latitude and longitude, use coordinates from additional_data only where they are missing in data
        data['latitude'] = data['latitude'].fillna(data['latitude_add'])
        data['longitude'] = data['longitude'].fillna(data['longitude_add'])

        # Now drop the additional columns as they are no longer needed
        data.drop(['latitude_add', 'longitude_add'], axis=1, inplace=True)

        # Fill remaining NaNs in other columns if necessary
        data.fillna('nan', inplace=True)

        print("Additional data added")  

        # Identify and drop duplicate entries in the 'postal_code' column
        data.drop_duplicates(subset='postal_code', keep='first', inplace=True)

        print("We have so much data: ", data.shape[0])  

    return data

In [ ]:
mapping_data = get_data("DE")
mapping_data.head()

In [15]:
def get_coordinates(mapping_data, postcode):
    entry = mapping_data[mapping_data['postal_code'] == postcode]
    if not entry.empty:
        # Get the longitude and latitude
        longitude = entry['longitude'].values[0]
        latitude = entry['latitude'].values[0]
        if pd.notna(longitude) and pd.notna(latitude):
            return (latitude, longitude)
        else:
            print(f"Coordinates are not available for postcode: {postcode}")
            return None
    else:
        print(f"Could not find coordinates for postcode: {postcode}")
        return None

In [16]:
def process_chunk(df):
    # Vorverarbeitung für fehlende Koordinaten
    missing_coords = df['Koordinate: Breitengrad (WGS84)'].isna() | df['Koordinate: Längengrad (WGS84)'].isna()
    missing_postcodes = df.loc[missing_coords, 'Postleitzahl']
    
    # Anzahl der Zeilen, die aktualisiert werden müssen
    total_missing = missing_coords.sum()

    # Hole Koordinaten für fehlende Postleitzahlen
    unique_postcodes = missing_postcodes.dropna().unique()
    print("Anzahl durchsuchte PLZs: ", len(unique_postcodes))
    coords_dict = {pc: get_coordinates(mapping_data, pc) for pc in unique_postcodes if pc}
    print("Anzahl gefundene Koordinaten: ", len(coords_dict))
    

    # Aktualisiere die Koordinaten im DataFrame
    for index, row in df[missing_coords].iterrows():
            
        coords = coords_dict.get(row['Postleitzahl'])
        if coords:
            df.at[index, 'Koordinate: Breitengrad (WGS84)'] = coords[0]
            df.at[index, 'Koordinate: Längengrad (WGS84)'] = coords[1]

    return df


In [17]:
def fill_missing_coordinates(df):
    updated_df = process_chunk(df)
    
    # Drop rows where 'Koordinate: Breitengrad (WGS84)' or 'Koordinate: Längengrad (WGS84)' is NaN
    updated_df.dropna(subset=['Koordinate: Breitengrad (WGS84)', 'Koordinate: Längengrad (WGS84)'], inplace=True)
    
    print("Gelöschte Zeilen: ", df.shape[0] - updated_df.shape[0])

    if not updated_df.empty:
        updated_df.to_csv("updated_data.csv", index=False, sep=";")
        print("Data processed and saved.")
    else:
        print("No updates for the data.")

    return updated_df


In [ ]:
num_unique_values = df["Postleitzahl"].nunique()
print(num_unique_values)

In [ ]:
updated_df = fill_missing_coordinates(df)

In [ ]:
print(f"Shape davor: {df.shape}")
print(f"Shape nur Einträge mit PLZ: {updated_df.shape}")
print(f"Anzahl gelöschte Zeilen: {df.shape[0] - updated_df.shape[0]}")

### Hinzufügen Landkreis Spalte

In [26]:
# Load GeoJSON 
with open('nrw-postleitzahlen.geojson', 'r') as f:
    landkreis_data = json.load(f)

In [28]:
# Create a mapping from plz_code to krs_name
plz_to_krs = {feature['properties']['plz_code']: feature['properties']['krs_name'] for feature in landkreis_data['features']}

In [ ]:
# Add Landkreis column and fill based on mapping table
updated_df['Landkreis'] = updated_df['Postleitzahl'].apply(lambda plz: plz_to_krs.get(plz, 'Unbekannt'))
updated_df.head()

In [ ]:
# Filter out rows where 'Landkreis' is None or missing
updated_df = updated_df[updated_df['Landkreis'].notna()]
updated_df.shape

In [ ]:
updated_df.columns

### Umwandlung in .geojson

In [39]:
# Stelle sicher, dass deine Koordinaten als Float vorliegen
updated_df['Koordinate: Breitengrad (WGS84)'] = pd.to_numeric(updated_df['Koordinate: Breitengrad (WGS84)'], errors='coerce')
updated_df['Koordinate: Längengrad (WGS84)'] = pd.to_numeric(updated_df['Koordinate: Längengrad (WGS84)'], errors='coerce')

df_clean = updated_df.drop(columns=['Inbetriebnahmedatum der Einheit formatted'])

# Erstelle ein GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_clean.drop(['Koordinate: Breitengrad (WGS84)', 'Koordinate: Längengrad (WGS84)'], axis=1),
    crs="EPSG:4326",
    geometry=[Point(xy) for xy in zip(df_clean['Koordinate: Längengrad (WGS84)'], df_clean['Koordinate: Breitengrad (WGS84)'])]
)

# Konvertiere das GeoDataFrame in GeoJSON
geojson = gdf.to_json()

# Save iti
with open('data_final_mit_Landkreis.geojson', 'w') as f:
    f.write(geojson)